In [1]:
import cv2
import os
import pickle
import face_recognition
import pandas as pd
import numpy as np
import cvzone
import datetime

In [2]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage 
# firebase_admin.delete_app(firebase_admin.get_app())
if not firebase_admin._apps:
    cred = credentials.Certificate(r"C:\Users\ayush\OneDrive\Pictures\realtimeattendence-firebase-adminsdk-cb6st-ddf1f7eda0.json")
    firebase_admin = firebase_admin.initialize_app(cred, {'databaseURL': 'https://realtimeattendence-default-rtdb.firebaseio.com/',
                            'storageBucket':'realtimeattendence.appspot.com'})

In [3]:
file=open(r"C:\Users\ayush\EncodeFilet.p","rb")
# load the known encode data with the student id
# pickle.load(file) returns the data in same format which has been dumped in the file without any changes
[studentId,encodeListKnown]=pickle.load(file)
print(studentId)
file.close()

['111', '112', '113']


In [4]:
# get all the changing screen images during the process of marking the attendence
# screen where we will display our data
imgModeList=[]
# get the list of images store in the data
modePath=r"C:\Users\ayush\OneDrive\Pictures\Modes"
modeList=os.listdir(modePath)
# read all the data using cv2 and store it in a list imgModeList
for i in modeList:
    temp=cv2.imread(modePath+"\\"+i)
    imgModeList.append(temp)

In [17]:
cap=cv2.VideoCapture(0)
# cap.set(3,width)
# cap.set(4,height)
cap.set(3,640)
cap.set(4,480)
background=cv2.imread(r"C:\Users\ayush\OneDrive\Pictures\background.png")
modeType=0

attendence=dict.fromkeys(studentId,0)

counter=0
sid=-1

bucket=storage.bucket()
imgStudent=[]
studentInfo=[]

while True:
    success,img=cap.read()
    
    imgS=cv2.resize(img,(0,0),None,0.25,0.25)
    imgS=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    background[162:162 + 480, 55:55 + 640] = img
    background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
    
#     background[162 + 480, 55 + 640] = img
#     background[44 + 633, 808 + 414] = imgModeList[modeType]
#     ValueError: could not broadcast input array from shape (480,640,3) into shape (640,3)
    
    faceCurFrame=face_recognition.face_locations(imgS)
    encodeCurFrame=face_recognition.face_encodings(imgS,faceCurFrame)
    
    if faceCurFrame:
        for encodeFace,faceLoc in zip(encodeCurFrame,faceCurFrame):
            
            matches=face_recognition.compare_faces(encodeListKnown,encodeFace,tolerance=0.5)
           # distance = sqrt(sum((a - b)^2)) it calculate the distance of two face encoding
           # the shortest distance of set of faces implies that the faces are similar
           # face_recognition.face_distance(listOfKnownFaces,unkownfaces)
           # it gives a booblean list whether the unkown faces mached with any of the faces
            faceDis=face_recognition.face_distance(encodeListKnown,encodeFace)
           # if the face_recognition.compare_faces() returns true for a face then we
           # calculate the distance between them is distance is also min compare to all other faces
           # then it is confirm that both the two faces are equal
            #print("matches",matches)
            #print("faceDis",faceDis)
            matchIndex=np.argmin(faceDis)
           #y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
    #         bbox=x1+55,y1+162,x2-x1,y2-y1
            #cv2.rectangle(background,(x1,y1),(x2+x1,y2+y1),(255,0,255),15)
            # rt= rectangle thickness
            #cvzone.cornerRect(background,bbox,rt=0)
            #print(faceLoc)
            if matches[matchIndex]:
                y1,x2,y2,x1=faceLoc
                bbox=x1+55,y1+162,x2-x1,y2-y1
                print("Known face detected")
                sid=studentId[matchIndex]
                print(sid)
                # draw rectangle around the face with rectangle thickness=0 it only display 4 _ around the corner
                cvzone.cornerRect(background,bbox,rt=0)
                if counter==0:
                    counter=1
                    modeType=1
                    # there is a time lag when the face is detected 
                    # so just for good ui we give loading on the screen with time lag of 1 ms
                    cvzone.putTextRect(background,"Loading",(275,400))
                    cv2.imshow("img",background)
                    cv2.waitKey(1)
            if counter!=0:
                if counter==1:
                    studentInfo=db.reference(f'Students/{sid}').get()
                    #print(studentInfo)

                    # get the img from the storage
                    blob=bucket.blob(f'{sid}')

                    datetimeObject=datetime.datetime.strptime(studentInfo['last_attendence_time'],
                                                   "%Y-%m-%d %H:%M:%S")
                    secondsElapsed=float((datetime.datetime.now()-datetimeObject).total_seconds())
                    print("+++++",secondsElapsed)
                    if secondsElapsed>30:
                        # standard meathod of conversion
                        array=np.frombuffer(blob.download_as_string(),np.uint8)
                        imgStudent=cv2.imdecode(array,cv2.COLOR_BGRA2BGR)
                        ref=db.reference(f'Students/{sid}')
                        studentInfo["total_attendence"]+=1
                        ref.child("total_attendence").set(studentInfo["total_attendence"])
                        curTime=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        ref.child("last_attendence_time").set(curTime)
                    else:
                        modeType=3
                        background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
                        imgStudent=[]
                        studentInfo=[]
                        counter=0
                if modeType!=3:    
                    if 10<counter<20:
                        modeType=2
                        background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]

                    if counter<=10:
                        modeType=1
                        background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
                        cv2.putText(background,str(studentInfo['total_attendence']),(861,125),cv2.FONT_HERSHEY_PLAIN,
                                   2,(255,255,255),1)
                        cv2.putText(background,str(studentInfo['major']),(1003,550),cv2.FONT_HERSHEY_PLAIN,
                                   0.6,(255,255,255),1)
                        cv2.putText(background,str(sid),(1003,493),cv2.FONT_HERSHEY_PLAIN,
                                   0.6,(255,255,255),1)
                        cv2.putText(background,str(studentInfo['starting_year']),(1120,625),cv2.FONT_HERSHEY_PLAIN,
                                    1,(100,100,100),1)
                        cv2.putText(background,str(studentInfo['year']+" Year"),(1007,625),cv2.FONT_HERSHEY_PLAIN,
                                   0.9,(100,100,100),1)
                        cv2.putText(background,str(studentInfo['sex']),(910,625),cv2.FONT_HERSHEY_PLAIN,
                                   1,(100,100,100),1)

                        (w,h),_=cv2.getTextSize(studentInfo['name'],cv2.FONT_HERSHEY_PLAIN,1,1)
                        offset=int((414-w)/2)
                        cv2.putText(background,str(studentInfo['name']),(808+offset,445),cv2.FONT_HERSHEY_PLAIN,
                                   1,(50,50,50),1)

                        background[175:175+216,909:909+216]=imgStudent
                counter+=1
                if counter>=20:
                    modeType=0
                    counter=0
                    imgStudent=[]
                    studentInfo=[]
                    background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
    else:
        # if no face is detected then we 
        modeType=0
        modeType=0
        counter=0
        imgStudent=[]
        studentInfo=[]
        background[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
                
    
    cv2.imshow("img",background)
    
    if cv2.waitKey(1) & 0xFF==ord('a'):
        break
cap.release()
cv2.destroyAllWindows()

Known face detected
113
+++++ 117.22694
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
+++++ 60.268949
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
Known face detected
113
+++++ 7.456545
Known face detected
113
+++++ 8.23331
Known face detected
113
+++++ 9.343531
+++++ 9.628328
+++++ 10.373575
+++++ 11.220385
+++++ 12.112426
+++++ 13.054762
Known face detected
112
+++++ 222.213515
Known face detected
112
Known face detected
112
Known face detected
112
Known face detected
112
Known face detected
112
Known face detected
112
Known face detected
112
Known face detected
112
+++++ 10.542274
Known face detected
112
+++++ 11.303108
Known face detected
112
+++++ 12.078204
+++++ 12.840997
Known face detected


##### face_recognition.api.compare_faces(known_face_encodings, face_encoding_to_check, tolerance=0.6)[source]
##### Compare a list of face encodings against a candidate encoding to see if they match.

##### Parameters:	
##### known_face_encodings – A list of known face encodings
##### face_encoding_to_check – A single face encoding to compare against the list
##### tolerance – How much distance between faces to consider it a match. Lower is more strict. 0.6 is typical best performance.
* https://face-recognition.readthedocs.io/en/latest/face_recognition.html

* face_recognition.api.face_encodings(face_image, known_face_locations=None, num_jitters=1, model='small')

In [6]:
print(datetime.datetime.strptime('2023-03-11 00:54:34',  "%Y-%m-%d %H:%M:%S"))
dir(datetime.datetime)

2023-03-11 00:54:34


['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'astimezone',
 'combine',
 'ctime',
 'date',
 'day',
 'dst',
 'fold',
 'fromisocalendar',
 'fromisoformat',
 'fromordinal',
 'fromtimestamp',
 'hour',
 'isocalendar',
 'isoformat',
 'isoweekday',
 'max',
 'microsecond',
 'min',
 'minute',
 'month',
 'now',
 'replace',
 'resolution',
 'second',
 'strftime',
 'strptime',
 'time',
 'timestamp',
 'timetuple',
 'timetz',
 'today',
 'toordinal',
 'tzinfo',
 'tzname',
 'utcfromtimestamp',
 'utcnow',
 'utcoffset',
 'utctimetuple',
 'weekday',
 'year']

In [7]:
dir(datetime)

['MAXYEAR',
 'MINYEAR',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'date',
 'datetime',
 'datetime_CAPI',
 'sys',
 'time',
 'timedelta',
 'timezone',
 'tzinfo']

In [8]:
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


'2023-03-13 05:16:00'

###### The first line of code, background[162:162 + 480, 55:55 + 640] = img, assigns the values of img to a rectangular region of background. Specifically, it selects a sub-region of background starting at row 162 and column 55, with a height of 480 pixels and a width of 640 pixels, and replaces the values in that sub-region with the corresponding values from img.

###### The second line of code, background[162 + 480, 55 + 640] = img, assigns the values of img to a single pixel in background. Specifically, it selects the pixel at row 642 (162+480) and column 695 (55+640), and replaces the value of that pixel with the corresponding value from img.

###### In summary, the first line of code assigns the values of img to a rectangular sub-region of background, while the second line of code assigns the values of img to a single pixel in background.

* The face_recognition.face_encodings() function from the face_recognition library is used to generate a numerical encoding (a feature vector) for a face in an image. This encoding can be used for tasks such as face recognition and face comparison.
* When calling the face_encodings() function, you can optionally pass in the face locations as an argument. The face location is a tuple of (top, right, bottom, left) coordinates that define the bounding box of the face in the image.

In [ ]:
# import face_recognition

# image = face_recognition.load_image_file("my_image.jpg")
# face_locations = face_recognition.face_locations(image)
# face_encodings = face_recognition.face_encodings(image, face_locations)

* In this example, face_locations is a list of tuples that contains the bounding box coordinates of all the faces in the image. By passing face_locations as an argument to face_encodings(), the function will only generate encodings for the faces in the specified locations.

* On the other hand, if you don't pass the face locations to the face_encodings() function, it will try to find all the faces in the image automatically using a built-in face detection algorithm.

In [ ]:
# import face_recognition

# image = face_recognition.load_image_file("my_image.jpg")
# face_encodings = face_recognition.face_encodings(image)


* In this case, the face_encodings() function will try to find all the faces in the image and generate encodings for each face it finds.

* So the main difference between calling face_encodings() with face locations and without face locations is that in the former, you are explicitly telling the function which faces to generate encodings for, whereas in the latter, the function will try to find all the faces in the image automatically.

# mode = 0 active
# mode =1 details
# mode=2 thank you
# mode=3 already marked